<a href="https://colab.research.google.com/github/Centrattic/SEESZooniverse/blob/main/data_processing/move_into_folders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
from tqdm import tqdm
from subprocess import call


In [ ]:
Zooni_data = pd.read_csv("drive/MyDrive/Data_Files/parsed_zooni_out.csv")

In [ ]:
Zooni_data.head()

,Unnamed: 0,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids,water,sky,land,img_name
0,0,501999199,MountainRobot03,2624165.0,a954d9bd77756da2e84f,24748,Choose the Land Cover Categories,6.9,2023-07-19 18:17:59 UTC,NaN,NaN,"{""source"":""api"",""session"":""1592f2abcf85083013e...","[{""task"":""T0"",""task_label"":""Select each of the...","{""90815996"":{""retired"":{""id"":118054626,""workfl...",90815996,0,1,1,3471352.jpg
1,1,501999214,MountainRobot03,2624165.0,a954d9bd77756da2e84f,24748,Choose the Land Cover Categories,6.9,2023-07-19 18:18:02 UTC,NaN,NaN,"{""source"":""api"",""session"":""1592f2abcf85083013e...","[{""task"":""T0"",""task_label"":""Select each of the...","{""90812812"":{""retired"":{""id"":118054627,""workfl...",90812812,0,1,1,3445155.jpg
2,2,501999223,MountainRobot03,2624165.0,a954d9bd77756da2e84f,24748,Choose the Land Cover Categories,6.9,2023-07-19 18:18:05 UTC,NaN,NaN,"{""source"":""api"",""session"":""1592f2abcf85083013e...","[{""task"":""T0"",""task_label"":""Select each of the...","{""90814635"":{""retired"":{""id"":118054630,""workfl...",90814635,0,0,1,3456233.jpg
3,3,501999240,MountainRobot03,2624165.0,a954d9bd77756da2e84f,24748,Choose the Land Cover Categories,6.9,2023-07-19 18:18:09 UTC,NaN,NaN,"{""source"":""api"",""session"":""1592f2abcf85083013e...","[{""task"":""T0"",""task_label"":""Select each of the...","{""90810735"":{""retired"":{""id"":118054631,""workfl...",90810735,0,1,1,3426903.jpg
4,4,501999254,MountainRobot03,2624165.0,a954d9bd77756da2e84f,24748,Choose the Land Cover Categories,6.9,2023-07-19 18:18:13 UTC,NaN,NaN,"{""source"":""api"",""session"":""1592f2abcf85083013e...","[{""task"":""T0"",""task_label"":""Select each of the...","{""90810670"":{""retired"":{""id"":118054636,""workfl...",90810670,1,0,1,3426742.jpg


In [ ]:
# don't run this. It deletes the whole dataset file: !rm -rf "drive/MyDrive/datasets/"

In [ ]:
def make_paths(base_path, split, ftr_name):
  positive_path = base_path + ftr_name + "/" + split + ftr_name
  negative_path = base_path + ftr_name + "/" + split + "no_" + ftr_name

  if not os.path.exists(positive_path):
    os.makedirs(positive_path)
  if not os.path.exists(negative_path):
    os.makedirs(negative_path)

In [ ]:
def df_to_folder(df, ftr_name, save_base_path, split,
                 store_path = "drive/MyDrive/Zooniverse_Images2/"):

  save_path_positive = save_base_path + ftr_name + "/" + split + ftr_name + "/"
  save_path_negative = save_base_path + ftr_name + "/" + split + "no_" + ftr_name + "/"

  for i in tqdm(range(len(df))):
    img_path = store_path + df['img_name'][i]
    if df[ftr_name][i] == 1:
      call(['cp', img_path, save_path_positive + df['img_name'][i]])
      # shutil.copy(store_path + df['img_name'][i], save_path_positive + df['img_name'][i])
    elif df[ftr_name][i] == 0:
      call(['cp', img_path, save_path_negative + df['img_name'][i]])
      # shutil.copy(store_path + df['img_name'][i], save_path_negative + df['img_name'][i])

In [ ]:
def move_to_folders(ftr_name, data, path): # either 'water' 'land' or 'sky'
  if not os.path.exists(path):
    os.makedirs(path)

  X_train, X_test, y_train, y_test = train_test_split(data, data[ftr_name],
                                                    test_size=0.2, stratify = data[ftr_name],
                                                    random_state=86)
  X_train, X_val, y_train, y_val = train_test_split(X_train, X_train[ftr_name],
                                                    test_size=0.125, stratify = X_train[ftr_name],
                                                    random_state=86)

  X_train.reset_index(drop=True, inplace=True)
  X_test.reset_index(drop=True, inplace=True)
  X_val.reset_index(drop=True, inplace=True)

  make_paths(path, "train/", ftr_name)
  make_paths(path, "val/", ftr_name)
  make_paths(path, "test/", ftr_name)

  df_to_folder(X_train, ftr_name, path, "train/")
  df_to_folder(X_val, ftr_name, path, "val/")
  df_to_folder(X_test, ftr_name, path, "test/")

In [ ]:
path = "drive/MyDrive/datasets/"

In [ ]:
move_to_folders("water", Zooni_data, path)

100%|██████████| 1180/1180 [05:23<00:00,  3.65it/s]


In [ ]:
move_to_folders("land", Zooni_data, path)

100%|██████████| 1180/1180 [00:14<00:00, 81.72it/s]


In [ ]:
move_to_folders("sky", Zooni_data, path)

100%|██████████| 1180/1180 [00:13<00:00, 85.97it/s]


In [ ]:
os.listdir(path)

['water', 'land', 'sky']